### Importar librerias y cargar csv

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

listing_clean = pd.read_csv('Recursos\listings_clean.csv')

### Gráfico Precio Medio por Tipo de propiedad

In [21]:
import plotly.express as px
# Calcular la media del precio por tipo de propiedad
mean_price = listing_clean.groupby('property_type')['price_eur'].mean().round(2).sort_values(ascending=False).head(10)
# Colores
colors = ['#FF3131' if i < 1 else '#36454F' for i in range(len(mean_price))]

fig = px.bar(mean_price, x=mean_price.index, y=mean_price.values, labels={'y':'Precio Medio', 'property_type':'Tipo de Propiedad'})
fig.update_layout(
    xaxis_title='Tipo de Propiedad',
    yaxis_title='Precio Medio',
    title='Precio Medio por Tipo de Propiedad'
)
fig.update_traces(marker_color=colors)
fig.show()


### Distribución todas las Valoraciones, Valoracion total y Precio

In [22]:
fig = px.scatter(listing_clean, x='review_scores_rating', y='number_of_reviews', color="price_eur",
                size="number_of_reviews", range_color=[0, 500], range_x=[4, 5],
                title="Distribución todas las Reviews, Valoración Total y Precio",
                labels={"review_scores_rating": "Valoración Total", "number_of_reviews": "Total Reviews", "price_eur": "Precio(€)"},)
fig.show()

### Distribución Numero de propiedades por total de comodidades y Precio Medio 

In [27]:
df_aux = listing_clean.copy()
df_aux['total_amenities'] = df_aux['amenities'].apply(lambda x: len(x.split(',')) if x != '[]' else 0)

mean_price_amenities=df_aux.groupby('total_amenities')['price_eur'].mean().round(2).reset_index(name='mean_price')
count_properties_amenities = df_aux.groupby('total_amenities')['price_eur'].count().reset_index(name='count_properties')

result_df = pd.merge(mean_price_amenities, count_properties_amenities, on='total_amenities')

fig = px.scatter(result_df, x='total_amenities', y='count_properties', color="mean_price", size="count_properties", title='Numero de propiedades por total de comodidades', range_color=[0, 500],
                labels={'mean_price': 'Precio Medio(€)', 'count_properties': 'Numero de propiedades', 'total_amenities': 'Total de comodidades'})
fig.update_layout(
    xaxis_title='Total de comodidades',
    yaxis_title='Numero de propiedades',
    title='Numero de propiedades por total de comodidades',
)
fig.show()


### Grafico barras Precio Medio(€) por número de Habitaciones con el Top 5 Tipos de Propiedad

In [28]:
df_aux = listing_clean.copy()
grouped_df = df_aux.groupby(['bedrooms', 'property_type']).agg(
    price_eur_mean=('price_eur', 'mean')
).reset_index()

top_5_property_types = df_aux['property_type'].value_counts().nlargest().index

filtered_grouped_df = grouped_df[grouped_df['property_type'].isin(top_5_property_types)]


final_filtered_df = filtered_grouped_df[(filtered_grouped_df['bedrooms'] >= 1)&(filtered_grouped_df['bedrooms'] <= 10)]
fig = px.bar(final_filtered_df,
            x="bedrooms",
            y="price_eur_mean",
            color="property_type",
            barmode="group",
            title="Precio Medio(€) por Número de Habitaciones con el Top 5 Tipos de Propiedad",
            labels={"bedrooms": "Número de Habitaciones", "price_eur_mean": "Precio Medio (EUR)", "property_type": "Tipo de propiedad"},
            category_orders={"bedrooms": sorted(final_filtered_df['bedrooms'].unique())})  # Ordenar las categorías de 'bedrooms'

fig.update_layout(
    plot_bgcolor="white",
    yaxis=dict(title='Precio Medio (EUR)', gridcolor='lightgrey'),
    xaxis=dict(title='Número de Habitaciones'),
    legend=dict(title='Tipo de Propiedad')
)
fig.show()

### Mapa Propiedades por Precio que tengan mas de 10 Reseñas

In [29]:
df_filtered = listing_clean.copy()
df_filtered['review_scores_rating'] = listing_clean[listing_clean["number_of_reviews"] > 10]["review_scores_rating"]

df_filtered = df_filtered.dropna(subset=["review_scores_rating"])
df_filtered = df_filtered.sort_values(by="review_scores_rating", ascending=True)
df_filtered['adjusted_size'] = df_filtered['price_eur'] + 35

fig = px.scatter_mapbox(df_filtered, lat="latitude", lon="longitude",
                        color="price_eur", size="adjusted_size",
                        size_max=55, 
                        animation_frame="review_scores_rating",
                        zoom=9, mapbox_style="open-street-map",
                        width=1200, height=800,
                        color_continuous_scale="viridis",
                        title="Propiedades por Precio que tengan mas de 10 Reseñas", range_color=[0, 500],
                        labels={"price_eur": "Precio (EUR)", "review_scores_rating": "Valoración Total"})


fig.show()


### TreeMap Distribución de Precio Medio por Barrio

In [26]:
listings_clean_df = listing_clean.copy()

listings_clean_df.dropna(subset=['neighbourhood', 'price_eur'], inplace=True)

price_mean_by_neighbourhood = listings_clean_df.groupby('neighbourhood', as_index=False)['price_eur'].mean().round(2)
price_mean_by_neighbourhood.rename(columns={'price_eur': 'price_mean_eur'}, inplace=True)

fig = px.treemap(price_mean_by_neighbourhood, 
                path=['neighbourhood'], 
                values='price_mean_eur',
                color='price_mean_eur', 
                color_continuous_scale='RdYlGn',
                title='Distribución de Precio Medio por Barrio',
                labels={'price_mean_eur': 'Precio Medio (€)'})
fig.update_traces(hovertemplate='Barrio: %{label}<br>Precio Medio (€): %{value}')
fig.show()